# ASCO 데이터셋을 RAW 데이터셋으로
- Hierarchical한 구조를 따름

In [4]:
root_dir = "/data_x/guro/DATA/ASCO"
save_dir = "/data_x/guro/DATA/ASCO_RAW"

In [17]:
import os
import json

all_data = []

for dir in os.listdir(root_dir):
  if dir != "readme.md":
    data_dir = os.path.join(root_dir, dir)
    section = None
    total_chunk = ""
    for file_name in os.listdir(data_dir):
      if file_name.endswith(".json"):
        file_path = os.path.join(data_dir, file_name)
        with open(file_path, "r") as f:
          data = json.load(f)
          metadata = data["meta"]
          contents = data["content"]
          for content in contents:
            chunk_section = content.get("section", None)
            chunk_text = content["chunk"]
            if chunk_section is not None:
              metadata["section"] = chunk_section
              if section != chunk_section:
                section = chunk_section
                total_chunk = chunk_text
              else:
                total_chunk += chunk_text + "\n"
                continue
            else:
              total_chunk = chunk_text
		  
            if chunk_section is not None:
              doc_id = metadata["doc_title"] + " - " + chunk_section
            else:
              doc_id = metadata["doc_title"]
            save_dict = {
              "doc_id": doc_id,
              "contents": total_chunk,
              "metadata": metadata,
            }
            all_data.append(save_dict)
len(all_data)


7445

In [18]:
all_data[0]

{'doc_id': 'Denosumab Prevents Bone Loss and Microarchitectural Deterioration in Premenopausal Women With Breast Cancer Receiving Estradiol Suppression Therapy: A Randomized Controlled Trial - Introduction',
 'contents': 'Estradiol plays an important role in breast cancer progression in women with estrogen receptor (ER)–positive breast cancer. Adjuvant endocrine therapies prevent breast cancer recurrence by competing with estradiol (selective estrogen receptor modulators, ie, tamoxifen) or by depleting circulating estradiol. In premenopausal women with ER-positive early-stage breast cancer at high risk of disease recurrence, maximal depletion of circulating estradiol by concurrent administration of gonadotropin-releasing hormone analogs, bilateral oophorectomy, or ovarian radiation, and AI improves breast cancer outcomes compared with standard monotherapy with tamoxifen and is now increasingly being prescribed for 5 or more years.1-3',
 'metadata': {'doc_title': 'Denosumab Prevents Bon

In [21]:
import pandas as pd

df = pd.DataFrame(all_data)
df.to_parquet(os.path.join(save_dir, "corpus.parquet"))

In [32]:
df_loaded = pd.read_parquet(os.path.join(save_dir, "raw_corpus.parquet"))
print(df_loaded.head())

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_loaded["doc_id"][0])
    print(df_loaded["contents"][0])
    print(df_loaded["doc_id"][1])
    print(df_loaded["contents"][1])
    print(df_loaded["doc_id"][2])
    print(df_loaded["contents"][2])


                                              doc_id  \
0  Denosumab Prevents Bone Loss and Microarchitec...   
1  Denosumab Prevents Bone Loss and Microarchitec...   
2  Denosumab Prevents Bone Loss and Microarchitec...   
3  Denosumab Prevents Bone Loss and Microarchitec...   
4  Denosumab Prevents Bone Loss and Microarchitec...   

                                            contents  \
0  Estradiol plays an important role in breast ca...   
1  This was a 12-month randomized, double-blind, ...   
2  Participants were stratified by lumbar spine B...   
3  Participants received 60 mg DMAB or matching P...   
4  Adverse events were documented at each visit a...   

                                            metadata  
0  {'doc_authors': 'Sabashini K. Ramchand, MBBS, ...  
1  {'doc_authors': 'Sabashini K. Ramchand, MBBS, ...  
2  {'doc_authors': 'Sabashini K. Ramchand, MBBS, ...  
3  {'doc_authors': 'Sabashini K. Ramchand, MBBS, ...  
4  {'doc_authors': 'Sabashini K. Ramchand, MBBS, ..

# Chunk by AutoRAG

In [3]:
# Parquet data 확인하는 명령어
import pyarrow.parquet as pq
import pandas as pd

# Load the parquet file
token_1000_200_parquet_file = "/data_x/guro/DATA/ASCO_RAW/CHUNKS/1.parquet"
token_500_100_parquet_file = "/data_x/guro/DATA/ASCO_RAW/CHUNKS/3.parquet"
token_300_50_parquet_file = "/data_x/guro/DATA/ASCO_RAW/CHUNKS/4.parquet"

# 첫번째 데이터들만 띄워서 보기
token_1000_200_df = pq.read_table(token_1000_200_parquet_file).to_pandas()
token_500_100_df = pq.read_table(token_500_100_parquet_file).to_pandas()
token_300_50_df = pq.read_table(token_300_50_parquet_file).to_pandas()

with pd.option_context('display.max_colwidth', None):
    print(token_1000_200_df["contents"][1])
    print(token_500_100_df["contents"][1])
    print(token_300_50_df["contents"][1])



file_name: 
 contents: This was a 12-month randomized, double-blind, PBO-controlled trial conducted at a single academic hospital. Premenopausal women, between age 18 and 55 years, were eligible for the study if they had confirmed ER-positive early-stage breast cancer and were going to receive treatment or had recently started treatment (<12 weeks before study enrollment) with combined OFS (gonadotropin-releasing hormone analog or bilateral oophorectomy) and AI.
file_name: 
 contents: This was a 12-month randomized, double-blind, PBO-controlled trial conducted at a single academic hospital. Premenopausal women, between age 18 and 55 years, were eligible for the study if they had confirmed ER-positive early-stage breast cancer and were going to receive treatment or had recently started treatment (<12 weeks before study enrollment) with combined OFS (gonadotropin-releasing hormone analog or bilateral oophorectomy) and AI.
file_name: 
 contents: This was a 12-month randomized, double-blin